### Lab: Value Iteration in a Grid World

### University of Virginia
### Reinforcement Learning
#### Last updated: December 11, 2023

---

#### Instructions:

Implement value iteration for a $4 \times 3$ gridworld environment. This will measure the value of each state. A robot in this world can make discrete moves: one step up, down, left or right. These actions are deterministic, meaning that the action selected will be taken with probability 1. There is a terminal state with reward +1 in the bottom right corner. All other states have reward 0. The discount factor is 0.9. Use tolerance $\theta=0.01$. Show all code and results.

**Note**: Do not use libraries from `networkx`, `gym`, `gymnasium` when solving this problem.

#### Total Points: 12

---

#### 1) **(POINTS: 2)** As part of your solution, create a GridWorld class with these attributes:

- `nrows` : number of rows in the grid
- `ncols` : number of columns in the grid

and these methods:

- `value_iteration()` with behavior described in [2] below
- `get_reward()` : given the agent row and column, return the reward

The class may include additional attributes and methods as well.

Create an instance using the class, and call `nrows`, `ncols`, and `get_reward()` to verify correctness.

You will not be graded on the implementation of `value_iteration()` for this problem.

#### 2) **(POINTS: 8)** Here, you will be graded on the implementation of `value_iteration()`.
Call `value_iteration()` to calculate and return the value function array. For each sweep over the states, have the function print out the intermediate array.


#### Enter all code here (you may also use multiple cells)

In [146]:
import numpy as np

#### 1) Create and test the class

In [147]:
class GridWorld():
    def __init__(self, nrows, ncols):
        self.nrows = nrows
        self.ncols = ncols
        self.values = np.zeros((nrows, ncols))
        self.actions = ['up', 'down', 'left', 'right']

    def value_iteration(self, theta=0.01, discount_factor=0.9, max_iter = 10):
      while True:
        max_delta = 0
        v = np.copy(self.values)

        for i in range(self.nrows):
          for j in range(self.ncols):
            # terminal state won't have value
            if i == self.nrows - 1 and j == self.ncols - 1:
              v[i,j] = 0
              continue

            # loop through all actions, finding best one from each state
            action_values = []
            for action in self.actions:
              if action == 'up':
                next_i = i - 1
                next_j = j
              elif action == 'down':
                next_i = i + 1
                next_j = j
              elif action == 'left':
                next_i = i
                next_j = j - 1
              elif action == 'right':
                next_i = i
                next_j = j + 1

              # check if action goes outside the grid, if so stay in state
              if next_i < 0 or next_i >= self.nrows or next_j < 0 or next_j >= self.ncols:
                action_values.append(self.get_reward(i, j) + discount_factor * v[i,j])
              # if next state is within grid, calculate new value
              else:
                action_values.append(self.get_reward(next_i, next_j) + discount_factor * v[next_i, next_j])

            # update state value with max action value
            v[i,j] = max(action_values)
            max_delta = max(max_delta, abs(v[i,j] - self.values[i,j]))

        if max_delta < theta:
          print("Threshold reached")
          break
        elif max_iter == 0:
          print("Max iterations reached")
          break
        else:
          max_iter -= 1
          self.values = v.copy()
          print("Iteration: ", 10 - max_iter)
          print(self.values)

      return self.values

    def get_reward(self, row, col):
        # if in bottom right, then reward is +1
        if row == self.nrows - 1 and col == self.ncols - 1:
            return 1
        else:
            return 0

In [148]:
gw = GridWorld(4, 3)
print(gw.nrows)
print(gw.ncols)
print(gw.get_reward(0, 0)) # should be no reward
print(gw.get_reward(2, 1)) # should be no reward
print(gw.get_reward(3, 2)) # bottom right, reward +1

4
3
0
0
1


#### 2) Run value iteration

In [149]:
gw.value_iteration(max_iter=10)

Iteration:  1
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
Iteration:  2
[[0.  0.  0. ]
 [0.  0.  0.9]
 [0.  0.9 1. ]
 [0.9 1.  0. ]]
Iteration:  3
[[0.   0.   0.81]
 [0.   0.81 0.9 ]
 [0.81 0.9  1.  ]
 [0.9  1.   0.  ]]
Iteration:  4
[[0.    0.729 0.81 ]
 [0.729 0.81  0.9  ]
 [0.81  0.9   1.   ]
 [0.9   1.    0.   ]]
Iteration:  5
[[0.6561 0.729  0.81  ]
 [0.729  0.81   0.9   ]
 [0.81   0.9    1.    ]
 [0.9    1.     0.    ]]
Threshold reached


array([[0.6561, 0.729 , 0.81  ],
       [0.729 , 0.81  , 0.9   ],
       [0.81  , 0.9   , 1.    ],
       [0.9   , 1.    , 0.    ]])

#### 3) **(POINTS: 2)** Based on the value function: After the agent has moved right or down, does it ever make sense for it to backtrack (move up or left)? Explain your reasoning.

After the agent has moved right or down, there is no need for it to ever backtrack. This is due to each value in the resulting value function being greater than all states above or to the left of it. This also makes sense intuitively as the best way to get to the bottom right would always be to move down or to the right, which we are trying to do.